# NovaPay Fraudulent Transaction Detection — Data Cleaning (Colab)

This notebook:
- Loads and **combines** `nova_pay_transcations.csv` and `nova_pay_fraud_boost.csv`
- Inspects schema and anomalies
- Cleans missing values intelligently
- Fixes inconsistent categories (channel, countries, KYC tier)
- Converts columns to proper data types
- Fixes invalid ranges (negative money values, score bounds, velocities)
- Saves a **final cleaned CSV** for EDA / modeling

> **Colab Tip:** Upload your CSV files (or mount Google Drive). The notebook supports both options.


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1) Import Required Libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
sns.set_theme(style="whitegrid")

print("✅ Libraries imported.")


✅ Libraries imported.


## 2) Data Loading

Choose **ONE** method below: upload files to Colab, or read from Google Drive.

### Load from Google Drive

In [18]:
# Uncomment if you want to load from Google Drive instead of uploading.
# from google.colab import drive
drive.mount('/content/drive')

# Example paths (edit to match your Drive folders):
df1 = pd.read_csv("/content/drive/MyDrive/nova_pay_fraud_boost.csv")
df2 = pd.read_csv("/content/drive/MyDrive/nova_pay_transcations.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load and Combine (stack rows)

In [19]:
# If you used Option A uploads, the files are in /content by default:
df1 = pd.read_csv("/content/drive/MyDrive/nova_pay_fraud_boost.csv")
df2 = pd.read_csv("/content/drive/MyDrive/nova_pay_transcations.csv")

# Combine them (stack rows)
df = pd.concat([df1, df2], ignore_index=True)

print("\n📌 Dataset Shape:", df.shape)
df.head()



📌 Dataset Shape: (11400, 26)


,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,exchange_rate_src_to_dest,device_id,new_device,ip_address,ip_country,location_mismatch,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,a3d2ad59-8e0f-4b33-b9cf-48bdca536e2f,22c3f4f2-23a3-4f40-adc0-307dff722216,2023-12-03 01:18:32.573611+00:00,UK,GBP,NGN,web,2267.24,2834.05,34.35,1388.888889,34864054-9a90-4560-bbdf-de7b39ff40c2,True,152.53.166.148,UK,False,0.929,low,24,0.027,1,0.312,8,11,0.22,1
1,ff87e7e7-7647-4059-8691-1a6d3e5251c2,db2b26f9-e12f-49d9-bf2f-73ae77a0d81f,2023-12-03 13:05:30.573611+00:00,US,USD,NGN,web,624.63,624.63,9.78,1111.111111,a1c8ade4-e978-471f-9cc4-98399747de6e,False,18.80.215.171,UK,True,1.000,low,14,0.566,0,0.311,6,7,0.00,1
2,5664b84f-2c90-4539-a5d9-6ec3bec881c5,74224638-f8fd-4ca1-91b0-244ffb6f7310,2023-12-05 03:38:56.573611+00:00,CA,CAD,INR,web,642.36,475.35,11.43,61.666667,64eb7b8c-c459-41cd-95ea-25f466bd8191,False,81.208.70.115,CA,False,1.000,low,65,0.386,1,0.425,7,8,0.12,1
3,771f1ad8-3902-4531-bb3c-6a4fa37ae438,93ca016d-93a2-4b86-b170-8871a9e4561e,2023-12-05 14:58:16.573611+00:00,US,USD,PHP,web,938.16,938.16,14.58,58.823529,3a60ff3e-1c6a-40b4-921b-c585632a8147,False,61.215.208.241,US,False,1.000,low,29,0.337,2,0.758,6,7,0.10,1
4,41dffb0b-2515-46ff-814c-724fa0f50c99,6ab3647c-b86e-4018-8fba-4077e0bf986a,2023-12-06 13:59:05.573611+00:00,US,USD,MXN,mobile,114.51,114.51,1.97,17.241379,ffe54f38-4df3-48f7-8909-a6457adfffc7,False,137.84.59.231,US,False,0.254,standard,1095,0.566,0,0.225,1,2,0.00,0


## 3) Data Information & Initial Inspection

In [20]:
print("\n📌 Dataset Info:")
df.info()

print("\n📌 Missing values (top 15):")
display(df.isnull().sum().sort_values(ascending=False).head(15))

print("\n📌 Duplicate rows:", df.duplicated().sum())



📌 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11400 entries, 0 to 11399
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   transaction_id             11400 non-null  object 
 1   customer_id                11400 non-null  object 
 2   timestamp                  11371 non-null  object 
 3   home_country               11400 non-null  object 
 4   source_currency            11400 non-null  object 
 5   dest_currency              11400 non-null  object 
 6   channel                    11400 non-null  object 
 7   amount_src                 11400 non-null  object 
 8   amount_usd                 11095 non-null  float64
 9   fee                        11105 non-null  float64
 10  exchange_rate_src_to_dest  11400 non-null  float64
 11  device_id                  11400 non-null  object 
 12  new_device                 11400 non-null  bool   
 13  ip_address                 11

,0
amount_usd,305
ip_address,305
ip_country,301
kyc_tier,300
device_trust_score,295
fee,295
timestamp,29
transaction_id,0
customer_id,0
home_country,0



📌 Duplicate rows: 200


## 4) Convert Columns to Proper Data Types

In [21]:
# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)

# Convert amount_src (object like '1,234.56') to float
if df["amount_src"].dtype == "object":
    df["amount_src"] = df["amount_src"].astype(str).str.replace(",", "", regex=False)
df["amount_src"] = pd.to_numeric(df["amount_src"], errors="coerce")

# Convert boolean columns to int
bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

print("✅ Data types converted.")
df.dtypes


✅ Data types converted.


,0
transaction_id,object
customer_id,object
timestamp,"datetime64[ns, UTC]"
home_country,object
source_currency,object
dest_currency,object
channel,object
amount_src,float64
amount_usd,float64
fee,float64


## 5) Drop Duplicates

In [22]:
dups = df.duplicated().sum()
print("📌 Duplicates found:", dups)

df = df.drop_duplicates().reset_index(drop=True)
print("✅ Duplicates dropped. New shape:", df.shape)


📌 Duplicates found: 200
✅ Duplicates dropped. New shape: (11200, 26)


## 6) Explore Categorical Disparities

In [23]:
categorical = df.select_dtypes(include="object")

id_columns = ["transaction_id", "customer_id", "device_id", "ip_address", "timestamp"]

for col in categorical.columns:
    if col not in id_columns:
        print(f"\nColumn: {col}")
        print("Unique values:", categorical[col].dropna().unique())
        print("Count of unique values:", categorical[col].nunique(dropna=True))



Column: home_country
Unique values: ['UK' 'US' 'CA' ' UK  ' ' US  ' 'unknown' ' CA  ']
Count of unique values: 7

Column: source_currency
Unique values: ['GBP' 'USD' 'CAD']
Count of unique values: 3

Column: dest_currency
Unique values: ['NGN' 'INR' 'PHP' 'MXN' 'CAD' 'USD' 'EUR' 'GBP' 'CNY']
Count of unique values: 9

Column: channel
Unique values: ['web' 'mobile' 'ATM' 'WEB' ' web  ' 'MOBILE' 'mobille' 'unknown'
 ' mobile  ' 'weeb' 'ATm' ' ATM  ']
Count of unique values: 12

Column: ip_country
Unique values: ['UK' 'CA' 'US' ' US  ' 'NAN' 'unknown' ' CA  ' ' UK  ' ' nan  ']
Count of unique values: 9

Column: kyc_tier
Unique values: ['low' 'standard' 'enhanced' ' standard  ' 'standrd' ' enhanced  '
 'STANDARD' 'NAN' 'unknown' 'enhancd' ' low  ' 'ENHANCED' 'LOW' ' nan  ']
Count of unique values: 14


## 7) Fix Inconsistent Categories (Countries, Channel, KYC Tier)

In [24]:
def _clean_text(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    if x.lower() in ["unknown", "nan", "none", "null", ""]:
        return np.nan
    return x

def fix_home_country(x):
    x = _clean_text(x)
    if pd.isna(x):
        return np.nan
    return str(x).strip().upper()

def fix_ip_country(x):
    x = _clean_text(x)
    if pd.isna(x):
        return np.nan
    return str(x).strip().upper()

def fix_channel(x):
    x = _clean_text(x)
    if pd.isna(x):
        return np.nan
    x_norm = str(x).strip().lower()

    if x_norm in ["atm", "at m", "atm  ", "at m  ", "atm_", "at m_", "atm-","atm."]:
        return "ATM"
    if x_norm in ["mobile", "mobille", " mobile", "mobile  ", "m0bile", "mobiile", " mobile  "]:
        return "MOBILE"
    if x_norm in ["web", "weeb", " web", "web  ", "w eb", "www", "website", " web  "]:
        return "WEB"

    if str(x).strip().upper() in ["ATM", "WEB", "MOBILE"]:
        return str(x).strip().upper()

    return str(x).strip().upper()

def fix_kyc_tier(x):
    x = _clean_text(x)
    if pd.isna(x):
        return np.nan
    x_norm = str(x).strip().lower()

    if x_norm in ["standard", "standrd", " standard", "standard  ", " standard  "]:
        return "STANDARD"
    if x_norm in ["enhanced", "enhancd", " enhanced", "enhanced  ", " enhanced  "]:
        return "ENHANCED"
    if x_norm in ["low", " low", "low  ", " low  "]:
        return "LOW"

    if str(x).strip().upper() in ["STANDARD", "ENHANCED", "LOW"]:
        return str(x).strip().upper()

    return str(x).strip().upper()

df["home_country"] = df["home_country"].apply(fix_home_country)
df["channel"] = df["channel"].apply(fix_channel)
df["ip_country"] = df["ip_country"].apply(fix_ip_country)
df["kyc_tier"] = df["kyc_tier"].apply(fix_kyc_tier)

print("✅ Category disparities fixed.")
df[["home_country","channel","ip_country","kyc_tier"]].head()


✅ Category disparities fixed.


,home_country,channel,ip_country,kyc_tier
0,UK,WEB,UK,LOW
1,US,WEB,UK,LOW
2,CA,WEB,CA,LOW
3,US,WEB,US,LOW
4,US,MOBILE,US,STANDARD


## 8) Fix Invalid Ranges

In [25]:
def fix_ranges(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Fix negative money values
    for col in ["amount_src", "amount_usd", "fee"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < 0, col] = np.nan

    # Clip score features to [0, 1]
    for col in ["ip_risk_score", "device_trust_score", "risk_score_internal", "corridor_risk"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df[col] = df[col].clip(lower=0.0, upper=1.0)

    # Velocities must be non-negative integers
    for col in ["txn_velocity_1h", "txn_velocity_24h"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < 0, col] = 0
            df[col] = df[col].fillna(0).astype(int)

    return df

df = fix_ranges(df)
print("✅ Ranges fixed.")
df[["amount_src","amount_usd","fee","ip_risk_score","device_trust_score","txn_velocity_1h","txn_velocity_24h"]].describe()


✅ Ranges fixed.


,amount_src,amount_usd,fee,ip_risk_score,device_trust_score,txn_velocity_1h,txn_velocity_24h
count,11100.000000,10900.000000,10810.000000,11200.000000,10910.000000,11200.000000,11200.000000
mean,444.331561,451.710904,100.139031,0.393969,0.655072,0.481696,0.732411
std,1374.173818,1401.028359,956.873963,0.261774,0.268407,1.521294,1.970835
min,7.230000,7.230000,0.500000,0.004000,0.000000,0.000000,0.000000
25%,92.202500,92.492500,2.410000,0.209000,0.515000,0.000000,0.000000
50%,160.490000,163.485000,3.530000,0.325000,0.654500,0.000000,0.000000
75%,297.890000,302.707500,5.590000,0.488000,0.894000,0.000000,0.000000
max,11942.890000,12498.570000,9999.990000,1.000000,0.999000,8.000000,11.000000


## 9) Handle Missing Values Intelligently

In [26]:
print("📌 Missing Values per Column (before):")
display(df.isnull().sum().sort_values(ascending=False))

# Drop rows missing timestamp
df = df.dropna(subset=["timestamp"]).reset_index(drop=True)

# Compute missing amount_usd
df["amount_usd"] = df["amount_usd"].fillna(df["amount_src"] * df["exchange_rate_src_to_dest"])

# Fill missing fee
df["fee"] = df["fee"].fillna(df["fee"].median())

# Fill missing IP address
df["ip_address"] = df["ip_address"].fillna("MISSING")

# Fill missing IP country
df["ip_country"] = df["ip_country"].fillna("Unknown")

# KYC tier missing
df["kyc_tier"] = df["kyc_tier"].fillna("Not_Verified")

# Device trust score
df["device_trust_score"] = df["device_trust_score"].fillna(df["device_trust_score"].median())

# Compute amount_src if missing
df["amount_src"] = df["amount_src"].fillna(df["amount_usd"] / df["exchange_rate_src_to_dest"])

# Drop rows where both amounts missing
df = df.dropna(subset=["amount_src", "amount_usd"], how="all").reset_index(drop=True)

# Fill missing home_country/channel
df["home_country"] = df["home_country"].fillna("Unknown")
df["channel"] = df["channel"].fillna("Unknown")

print("✅ Missing values handled.")
print("\n📌 Missing Values per Column (after):")
display(df.isnull().sum().sort_values(ascending=False).head(10))


📌 Missing Values per Column (before):


,0
fee,390
ip_country,331
kyc_tier,329
ip_address,300
amount_usd,300
device_trust_score,290
amount_src,100
timestamp,60
channel,37
home_country,32


✅ Missing values handled.

📌 Missing Values per Column (after):


,0
transaction_id,0
customer_id,0
timestamp,0
home_country,0
source_currency,0
dest_currency,0
channel,0
amount_src,0
amount_usd,0
fee,0


## 10) Final Dataset Check

In [27]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11137 entries, 0 to 11136
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   transaction_id             11137 non-null  object             
 1   customer_id                11137 non-null  object             
 2   timestamp                  11137 non-null  datetime64[ns, UTC]
 3   home_country               11137 non-null  object             
 4   source_currency            11137 non-null  object             
 5   dest_currency              11137 non-null  object             
 6   channel                    11137 non-null  object             
 7   amount_src                 11137 non-null  float64            
 8   amount_usd                 11137 non-null  float64            
 9   fee                        11137 non-null  float64            
 10  exchange_rate_src_to_dest  11137 non-null  float64            
 11  de

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,exchange_rate_src_to_dest,device_id,new_device,ip_address,ip_country,location_mismatch,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,a3d2ad59-8e0f-4b33-b9cf-48bdca536e2f,22c3f4f2-23a3-4f40-adc0-307dff722216,2023-12-03 01:18:32.573611+00:00,UK,GBP,NGN,WEB,2267.24,2834.05,34.35,1388.888889,34864054-9a90-4560-bbdf-de7b39ff40c2,1,152.53.166.148,UK,0,0.929,LOW,24,0.027,1,0.312,8,11,0.22,1
1,ff87e7e7-7647-4059-8691-1a6d3e5251c2,db2b26f9-e12f-49d9-bf2f-73ae77a0d81f,2023-12-03 13:05:30.573611+00:00,US,USD,NGN,WEB,624.63,624.63,9.78,1111.111111,a1c8ade4-e978-471f-9cc4-98399747de6e,0,18.80.215.171,UK,1,1.000,LOW,14,0.566,0,0.311,6,7,0.00,1
2,5664b84f-2c90-4539-a5d9-6ec3bec881c5,74224638-f8fd-4ca1-91b0-244ffb6f7310,2023-12-05 03:38:56.573611+00:00,CA,CAD,INR,WEB,642.36,475.35,11.43,61.666667,64eb7b8c-c459-41cd-95ea-25f466bd8191,0,81.208.70.115,CA,0,1.000,LOW,65,0.386,1,0.425,7,8,0.12,1
3,771f1ad8-3902-4531-bb3c-6a4fa37ae438,93ca016d-93a2-4b86-b170-8871a9e4561e,2023-12-05 14:58:16.573611+00:00,US,USD,PHP,WEB,938.16,938.16,14.58,58.823529,3a60ff3e-1c6a-40b4-921b-c585632a8147,0,61.215.208.241,US,0,1.000,LOW,29,0.337,2,0.758,6,7,0.10,1
4,41dffb0b-2515-46ff-814c-724fa0f50c99,6ab3647c-b86e-4018-8fba-4077e0bf986a,2023-12-06 13:59:05.573611+00:00,US,USD,MXN,MOBILE,114.51,114.51,1.97,17.241379,ffe54f38-4df3-48f7-8909-a6457adfffc7,0,137.84.59.231,US,0,0.254,STANDARD,1095,0.566,0,0.225,1,2,0.00,0


## 11) Save the Cleaned Dataset

In [28]:
from google.colab import files

output_path = "/content/Nova_cleaned_df.csv"
df.to_csv(output_path, index=False)

print("✅ Saved cleaned dataset to:", output_path)
files.download(output_path)


✅ Saved cleaned dataset to: /content/Nova_cleaned_df.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>